In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns
 
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_dir = '../input/lish-moa/'
os.listdir(data_dir)

['lish-moa.zip',
 'sample_submission.csv',
 'test_features.csv',
 'train_features.csv',
 'train_targets_nonscored.csv',
 'train_targets_scored.csv',
 'train_features_dummy.csv',
 'test_features_dummy.csv',
 'train_drug.csv',
 'main_predictors.json',
 '.ipynb_checkpoints']

In [3]:
train_features = pd.read_csv(data_dir + 'train_features.csv')
train_targets_scored = pd.read_csv(data_dir + 'train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir + 'train_targets_nonscored.csv')
train_drug = pd.read_csv(data_dir + 'train_drug.csv')
test_features = pd.read_csv(data_dir + 'test_features.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

print('train_features: {}'.format(train_features.shape))
print('train_targets_scored: {}'.format(train_targets_scored.shape))
print('train_targets_nonscored: {}'.format(train_targets_nonscored.shape))
print('train_drug: {}'.format(train_drug.shape))
print('test_features: {}'.format(test_features.shape))
print('sample_submission: {}'.format(sample_submission.shape))

train_features: (23814, 876)
train_targets_scored: (23814, 207)
train_targets_nonscored: (23814, 403)
train_drug: (23814, 2)
test_features: (3982, 876)
sample_submission: (3982, 207)


In [4]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

print('GENES: {}'.format(GENES[:10]))
print('CELLS: {}'.format(CELLS[:10]))

GENES: ['g-0', 'g-1', 'g-2', 'g-3', 'g-4', 'g-5', 'g-6', 'g-7', 'g-8', 'g-9']
CELLS: ['c-0', 'c-1', 'c-2', 'c-3', 'c-4', 'c-5', 'c-6', 'c-7', 'c-8', 'c-9']


In [5]:
# !pip install /kaggle/input/iterative-stratification/iterative-stratification-master/
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# RankGauss

In [6]:
import pickle
# RankGauss - transform to Gauss
QT_scaler = []
for col in (GENES + CELLS):
#     transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
#     vec_len = len(train_features[col].values)
#     vec_len_test = len(test_features[col].values)
#     raw_vec = train_features[col].values.reshape(vec_len, 1)
#     transformer.fit(raw_vec)
    
    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")   # from optimal commit 9
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)
    QT_scaler.append([col,transformer])
    
    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
    
    
file = open('QT_scaler.pkl', 'wb')
pickle.dump(QT_scaler, file)
file.close()



In [7]:
SEED_VALUE = 42

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=SEED_VALUE)

# PCA features + Existing features

In [8]:
# GENES
n_comp = 600

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])

PCA_transfer1 = (PCA(n_components=n_comp, random_state=42).fit(data[GENES]))
data2 = PCA_transfer1.transform(data[GENES])

file = open('PCA_transfer1.pkl', 'wb')
pickle.dump(PCA_transfer1, file)
file.close()


# data2 = (PCA(n_components=n_comp, random_state=SEED_VALUE).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

train_features: (23814, 1476)
test_features: (3982, 1476)


In [9]:
# CELLS
n_comp = 50

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])

PCA_transfer2 = (PCA(n_components=n_comp, random_state=42).fit(data[CELLS]))
data2 = PCA_transfer2.transform(data[CELLS])

file = open('PCA_transfer2.pkl', 'wb')
pickle.dump(PCA_transfer2, file)
file.close()


# data2 = (PCA(n_components=n_comp, random_state=SEED_VALUE).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

train_features: (23814, 1526)
test_features: (3982, 1526)


# feature Selection using Variance Encoding

In [10]:
from sklearn.feature_selection import VarianceThreshold

var_thresh = VarianceThreshold(0.8)
data = train_features.append(test_features)

var_thresh.fit(data.iloc[:, 4:])
file = open('var_thresh.pkl', 'wb')
pickle.dump(var_thresh, file)
file.close()

data_transformed = var_thresh.transform(data.iloc[:, 4:])

# data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]

train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)

test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

print('train_features: {}'.format(train_features.shape))
print('test_features: {}'.format(test_features.shape))

train_features: (23814, 1040)
test_features: (3982, 1040)


In [11]:
train_drug.head()

,sig_id,drug_id
0,id_000644bb2,b68db1d53
1,id_000779bfc,df89a8e5a
2,id_000a6266a,18bb41b2c
3,id_0015fd391,8c7f86626
4,id_001626bd3,7cbed3131


In [12]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored, on='sig_id')
train = train.merge(train_drug, on='sig_id')
train = train[train['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)

In [13]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [14]:
train.head()

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,...,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor,drug_id
0,id_000644bb2,24,D1,1.134849,0.907687,-0.416385,-0.966814,-0.254723,-1.017473,-1.364787,...,0,0,0,0,0,0,0,0,0,b68db1d53
1,id_000779bfc,72,D1,0.119282,0.681738,0.272399,0.080113,1.205169,0.686517,0.313396,...,0,0,0,0,0,0,0,0,0,df89a8e5a
2,id_000a6266a,48,D1,0.779973,0.946463,1.425350,-0.132928,-0.006122,1.492493,0.235577,...,0,0,0,0,0,0,0,0,0,18bb41b2c
3,id_0015fd391,48,D1,-0.734910,-0.274641,-0.438509,0.759097,2.346330,-0.858153,-2.288417,...,0,0,0,0,0,0,0,0,0,8c7f86626
4,id_001626bd3,72,D2,-0.452718,-0.477513,0.972316,0.970731,1.463427,-0.869555,-0.375501,...,0,0,0,0,0,0,0,0,0,7cbed3131


In [15]:
target_cols = [x for x in train_targets_scored.columns if x != 'sig_id']
aux_target_cols = [x for x in train_targets_nonscored.columns if x != 'sig_id']
all_target_cols = target_cols + aux_target_cols

num_targets = len(target_cols)
num_aux_targets = len(aux_target_cols)
num_all_targets = len(all_target_cols)

print('num_targets: {}'.format(num_targets))
print('num_aux_targets: {}'.format(num_aux_targets))
print('num_all_targets: {}'.format(num_all_targets))

num_targets: 206
num_aux_targets: 402
num_all_targets: 608


In [16]:
print(train.shape)
print(test.shape)
print(sample_submission.shape)

(21948, 1648)
(3624, 1039)
(3982, 207)


# Dataset Classes

In [17]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)
        }
        
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }

        return dct

In [18]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    return final_loss

def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    return preds

In [19]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1

        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
            
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

# Model

In [20]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model, self).__init__()
        self.hidden_size = [1500, 1250, 1000, 750]
        self.dropout_value = [0.5, 0.35, 0.3, 0.25]

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, self.hidden_size[0])
        
        self.batch_norm2 = nn.BatchNorm1d(self.hidden_size[0])
        self.dropout2 = nn.Dropout(self.dropout_value[0])
        self.dense2 = nn.Linear(self.hidden_size[0], self.hidden_size[1])

        self.batch_norm3 = nn.BatchNorm1d(self.hidden_size[1])
        self.dropout3 = nn.Dropout(self.dropout_value[1])
        self.dense3 = nn.Linear(self.hidden_size[1], self.hidden_size[2])

        self.batch_norm4 = nn.BatchNorm1d(self.hidden_size[2])
        self.dropout4 = nn.Dropout(self.dropout_value[2])
        self.dense4 = nn.Linear(self.hidden_size[2], self.hidden_size[3])

        self.batch_norm5 = nn.BatchNorm1d(self.hidden_size[3])
        self.dropout5 = nn.Dropout(self.dropout_value[3])
        self.dense5 = nn.utils.weight_norm(nn.Linear(self.hidden_size[3], num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.leaky_relu(self.dense3(x))

        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = F.leaky_relu(self.dense4(x))

        x = self.batch_norm5(x)
        x = self.dropout5(x)
        x = self.dense5(x)
        return x
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
            
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))    

In [21]:
class FineTuneScheduler:
    def __init__(self, epochs):
        self.epochs = epochs
        self.epochs_per_step = 0
        self.frozen_layers = []

    def copy_without_top(self, model, num_features, num_targets, num_targets_new):
        self.frozen_layers = []

        model_new = Model(num_features, num_targets)
        model_new.load_state_dict(model.state_dict())

        # Freeze all weights
        for name, param in model_new.named_parameters():
            layer_index = name.split('.')[0][-1]

            if layer_index == 5:
                continue

            param.requires_grad = False

            # Save frozen layer names
            if layer_index not in self.frozen_layers:
                self.frozen_layers.append(layer_index)

        self.epochs_per_step = self.epochs // len(self.frozen_layers)

        # Replace the top layers with another ones
        model_new.batch_norm5 = nn.BatchNorm1d(model_new.hidden_size[3])
        model_new.dropout5 = nn.Dropout(model_new.dropout_value[3])
        model_new.dense5 = nn.utils.weight_norm(nn.Linear(model_new.hidden_size[-1], num_targets_new))
        model_new.to(DEVICE)
        return model_new

    def step(self, epoch, model):
        if len(self.frozen_layers) == 0:
            return

        if epoch % self.epochs_per_step == 0:
            last_frozen_index = self.frozen_layers[-1]
            
            # Unfreeze parameters of the last frozen layer
            for name, param in model.named_parameters():
                layer_index = name.split('.')[0][-1]

                if layer_index == last_frozen_index:
                    param.requires_grad = True

            del self.frozen_layers[-1]  # Remove the last layer as unfrozen

# Preprocessing steps

In [22]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [23]:
feature_cols = [c for c in process_data(train).columns if c not in all_target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold', 'sig_id', 'drug_id']]
num_features = len(feature_cols)
num_features

1041

In [24]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 24
BATCH_SIZE = 128

WEIGHT_DECAY = {'ALL_TARGETS': 1e-5, 'SCORED_ONLY': 3e-6}
MAX_LR = {'ALL_TARGETS': 1e-2, 'SCORED_ONLY': 3e-3}
DIV_FACTOR = {'ALL_TARGETS': 1e3, 'SCORED_ONLY': 1e2}
PCT_START = 0.1

In [25]:
# Show model architecture
model = Model(num_features, num_all_targets)
model

Model(
  (batch_norm1): BatchNorm1d(1041, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=1041, out_features=1500, bias=True)
  (batch_norm2): BatchNorm1d(1500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=1500, out_features=1250, bias=True)
  (batch_norm3): BatchNorm1d(1250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout(p=0.35, inplace=False)
  (dense3): Linear(in_features=1250, out_features=1000, bias=True)
  (batch_norm4): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout4): Dropout(p=0.3, inplace=False)
  (dense4): Linear(in_features=1000, out_features=750, bias=True)
  (batch_norm5): BatchNorm1d(750, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout5): Dropout(p=0.25, inplace=False)
  (dense5): Linear(in_features=750, out_features=608, 

# Single fold training

In [26]:
from sklearn.model_selection import KFold

def make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH):
    vc = train.drug_id.value_counts()
    vc1 = vc.loc[vc <= DRUG_THRESH].index.sort_values()
    vc2 = vc.loc[vc > DRUG_THRESH].index.sort_values()

    for seed_id in range(SEEDS):
        kfold_col = 'kfold_{}'.format(seed_id)
        
        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}
        dct2 = {}

        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.groupby('drug_id')[target_cols].mean().loc[vc1]

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.index[idxV].values}
            dct1.update(dd)

        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.loc[train.drug_id.isin(vc2)].reset_index(drop=True)

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)

        # ASSIGN FOLDS
        train[kfold_col] = train.drug_id.map(dct1)
        train.loc[train[kfold_col].isna(), kfold_col] = train.loc[train[kfold_col].isna(), 'sig_id'].map(dct2)
        train[kfold_col] = train[kfold_col].astype('int8')
        
    return train

SEEDS = 7
NFOLDS = 7
DRUG_THRESH = 18

# train = make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH)
# train.head()

In [27]:
train.head()

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,...,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor,drug_id
0,id_000644bb2,24,D1,1.134849,0.907687,-0.416385,-0.966814,-0.254723,-1.017473,-1.364787,...,0,0,0,0,0,0,0,0,0,b68db1d53
1,id_000779bfc,72,D1,0.119282,0.681738,0.272399,0.080113,1.205169,0.686517,0.313396,...,0,0,0,0,0,0,0,0,0,df89a8e5a
2,id_000a6266a,48,D1,0.779973,0.946463,1.425350,-0.132928,-0.006122,1.492493,0.235577,...,0,0,0,0,0,0,0,0,0,18bb41b2c
3,id_0015fd391,48,D1,-0.734910,-0.274641,-0.438509,0.759097,2.346330,-0.858153,-2.288417,...,0,0,0,0,0,0,0,0,0,8c7f86626
4,id_001626bd3,72,D2,-0.452718,-0.477513,0.972316,0.970731,1.463427,-0.869555,-0.375501,...,0,0,0,0,0,0,0,0,0,7cbed3131


In [28]:

mskf = MultilabelStratifiedKFold(n_splits=7)
train['kfold'] = 0
for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=train[train_targets_scored.columns])):
    train.loc[v_idx, 'kfold'] = int(f)

train['kfold'] = train['kfold'].astype(int)
train.head()

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,...,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor,drug_id,kfold
0,id_000644bb2,24,D1,1.134849,0.907687,-0.416385,-0.966814,-0.254723,-1.017473,-1.364787,...,0,0,0,0,0,0,0,0,b68db1d53,5
1,id_000779bfc,72,D1,0.119282,0.681738,0.272399,0.080113,1.205169,0.686517,0.313396,...,0,0,0,0,0,0,0,0,df89a8e5a,0
2,id_000a6266a,48,D1,0.779973,0.946463,1.425350,-0.132928,-0.006122,1.492493,0.235577,...,0,0,0,0,0,0,0,0,18bb41b2c,6
3,id_0015fd391,48,D1,-0.734910,-0.274641,-0.438509,0.759097,2.346330,-0.858153,-2.288417,...,0,0,0,0,0,0,0,0,8c7f86626,0
4,id_001626bd3,72,D2,-0.452718,-0.477513,0.972316,0.970731,1.463427,-0.869555,-0.375501,...,0,0,0,0,0,0,0,0,7cbed3131,4


In [29]:
def run_training(fold_id, seed_id):
    seed_everything(seed_id)
    
    train_ = process_data(train)
    test_ = process_data(test)
    
    kfold_col = f'kfold'
    trn_idx = train_[train_[kfold_col] != fold_id].index
    val_idx = train_[train_[kfold_col] == fold_id].index
    
    train_df = train_[train_[kfold_col] != fold_id].reset_index(drop=True)
    valid_df = train_[train_[kfold_col] == fold_id].reset_index(drop=True)
    
    def train_model(model, tag_name, target_cols_now, fine_tune_scheduler=None):
        x_train, y_train  = train_df[feature_cols].values, train_df[target_cols_now].values
        x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols_now].values
        
        train_dataset = MoADataset(x_train, y_train)
        valid_dataset = MoADataset(x_valid, y_valid)

        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=WEIGHT_DECAY[tag_name])
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
                                                  steps_per_epoch=len(trainloader),
                                                  pct_start=PCT_START,
                                                  div_factor=DIV_FACTOR[tag_name], 
                                                  max_lr=MAX_LR[tag_name],
                                                  epochs=EPOCHS)
        
        loss_fn = nn.BCEWithLogitsLoss()
        loss_tr = SmoothBCEwLogits(smoothing=0.001)

        oof = np.zeros((len(train), len(target_cols_now)))
        best_loss = np.inf
        
        for epoch in range(EPOCHS):
            if fine_tune_scheduler is not None:
                fine_tune_scheduler.step(epoch, model)

            train_loss = train_fn(model, optimizer, scheduler, loss_tr, trainloader, DEVICE)
            valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
            print(f"SEED: {seed_id}, FOLD: {fold_id}, {tag_name}, EPOCH: {epoch}, train_loss: {train_loss:.6f}, valid_loss: {valid_loss:.6f}")

            if np.isnan(valid_loss):
                break
            
            if valid_loss < best_loss:
                best_loss = valid_loss
                oof[val_idx] = valid_preds
                torch.save(model.state_dict(), f"{tag_name}_FOLD{fold_id}_SEED{seed_id}.pth")

        return oof

    fine_tune_scheduler = FineTuneScheduler(EPOCHS)

    pretrained_model = Model(num_features, num_all_targets)
    pretrained_model.to(DEVICE)

    # Train on scored + nonscored targets
    train_model(pretrained_model, 'ALL_TARGETS', all_target_cols)

    # Load the pretrained model with the best loss
    pretrained_model = Model(num_features, num_all_targets)
    pretrained_model.load_state_dict(torch.load(f"ALL_TARGETS_FOLD{fold_id}_SEED{seed_id}.pth"))
    pretrained_model.to(DEVICE)

    # Copy model without the top layer
    final_model = fine_tune_scheduler.copy_without_top(pretrained_model, num_features, num_all_targets, num_targets)

    # Fine-tune the model on scored targets only
    oof = train_model(final_model, 'SCORED_ONLY', target_cols, fine_tune_scheduler)

    # Load the fine-tuned model with the best loss
    model = Model(num_features, num_targets)
    model.load_state_dict(torch.load(f"SCORED_ONLY_FOLD{fold_id}_SEED{seed_id}.pth"))
    model.to(DEVICE)

    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    predictions = np.zeros((len(test_), num_targets))
    predictions = inference_fn(model, testloader, DEVICE)
    return oof, predictions

In [30]:
def run_k_fold(NFOLDS, seed_id):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold_id in range(NFOLDS):
        oof_, pred_ = run_training(fold_id, seed_id)
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [31]:
from time import time

# Averaging on multiple SEEDS
SEED = [0, 1, 2, 3, 4, 5, 6]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

time_begin = time()

for seed_id in SEED:
    oof_, predictions_ = run_k_fold(NFOLDS, seed_id)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

time_diff = time() - time_begin

train[target_cols] = oof


SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 0, train_loss: 0.496843, valid_loss: 0.022465
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 1, train_loss: 0.015825, valid_loss: 0.010359
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 2, train_loss: 0.013468, valid_loss: 0.009927
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 3, train_loss: 0.013041, valid_loss: 0.009689
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 4, train_loss: 0.012958, valid_loss: 0.009808
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 5, train_loss: 0.012871, valid_loss: 0.009569
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 6, train_loss: 0.012861, valid_loss: 0.009601
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 7, train_loss: 0.012839, valid_loss: 0.009513
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 8, train_loss: 0.012843, valid_loss: 0.009539
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 9, train_loss: 0.012841, valid_loss: 0.009601
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 10, train_loss: 0.012819, valid_loss: 0.009484
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 11, train_loss: 0.012773, valid_loss:

SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 1, train_loss: 0.015517, valid_loss: 0.010627
SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 2, train_loss: 0.013426, valid_loss: 0.009718
SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 3, train_loss: 0.014443, valid_loss: 0.009527
SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 4, train_loss: 0.012881, valid_loss: 0.009435
SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 5, train_loss: 0.012838, valid_loss: 0.009571
SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 6, train_loss: 0.012836, valid_loss: 0.009407
SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 7, train_loss: 0.012827, valid_loss: 0.009382
SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 8, train_loss: 0.012849, valid_loss: 0.009548
SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 9, train_loss: 0.012837, valid_loss: 0.009544
SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 10, train_loss: 0.012827, valid_loss: 0.009381
SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 11, train_loss: 0.012799, valid_loss: 0.009494
SEED: 0, FOLD: 2, ALL_TARGETS, EPOCH: 12, train_loss: 0.012786, valid_loss

SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 2, train_loss: 0.013538, valid_loss: 0.009619
SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 3, train_loss: 0.013004, valid_loss: 0.009773
SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 4, train_loss: 0.012852, valid_loss: 0.009443
SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 5, train_loss: 0.012931, valid_loss: 0.009557
SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 6, train_loss: 0.012864, valid_loss: 0.009521
SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 7, train_loss: 0.012851, valid_loss: 0.009594
SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 8, train_loss: 0.012806, valid_loss: 0.009482
SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 9, train_loss: 0.012834, valid_loss: 0.009502
SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 10, train_loss: 0.012798, valid_loss: 0.009530
SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 11, train_loss: 0.012798, valid_loss: 0.009481
SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 12, train_loss: 0.012789, valid_loss: 0.009430
SEED: 0, FOLD: 4, ALL_TARGETS, EPOCH: 13, train_loss: 0.012762, valid_los

SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 3, train_loss: 0.013034, valid_loss: 0.009474
SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 4, train_loss: 0.013195, valid_loss: 0.009558
SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 5, train_loss: 0.012957, valid_loss: 0.009502
SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 6, train_loss: 0.012906, valid_loss: 0.009450
SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 7, train_loss: 0.012893, valid_loss: 0.009402
SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 8, train_loss: 0.012864, valid_loss: 0.009539
SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 9, train_loss: 0.012842, valid_loss: 0.009318
SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 10, train_loss: 0.012806, valid_loss: 0.009412
SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 11, train_loss: 0.012795, valid_loss: 0.009442
SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 12, train_loss: 0.012801, valid_loss: 0.009410
SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 13, train_loss: 0.012744, valid_loss: 0.009398
SEED: 0, FOLD: 6, ALL_TARGETS, EPOCH: 14, train_loss: 0.012735, valid_lo

SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 4, train_loss: 0.013140, valid_loss: 0.009351
SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 5, train_loss: 0.012950, valid_loss: 0.009443
SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 6, train_loss: 0.012912, valid_loss: 0.009422
SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 7, train_loss: 0.012896, valid_loss: 0.009439
SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 8, train_loss: 0.012899, valid_loss: 0.009446
SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 9, train_loss: 0.012881, valid_loss: 0.009355
SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 10, train_loss: 0.012885, valid_loss: 0.009415
SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 11, train_loss: 0.012892, valid_loss: 0.009355
SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 12, train_loss: 0.012847, valid_loss: 0.009266
SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 13, train_loss: 0.012793, valid_loss: 0.009309
SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 14, train_loss: 0.012765, valid_loss: 0.009267
SEED: 1, FOLD: 1, ALL_TARGETS, EPOCH: 15, train_loss: 0.012740, valid_l

...

SEED: 5, FOLD: 5, ALL_TARGETS, EPOCH: 20, train_loss: 0.012244, valid_loss: 0.008955
SEED: 5, FOLD: 5, ALL_TARGETS, EPOCH: 21, train_loss: 0.012141, valid_loss: 0.008922
SEED: 5, FOLD: 5, ALL_TARGETS, EPOCH: 22, train_loss: 0.012045, valid_loss: 0.008885
SEED: 5, FOLD: 5, ALL_TARGETS, EPOCH: 23, train_loss: 0.011988, valid_loss: 0.008889
SEED: 5, FOLD: 5, SCORED_ONLY, EPOCH: 0, train_loss: 0.643474, valid_loss: 0.267901
SEED: 5, FOLD: 5, SCORED_ONLY, EPOCH: 1, train_loss: 0.059249, valid_loss: 0.019932
SEED: 5, FOLD: 5, SCORED_ONLY, EPOCH: 2, train_loss: 0.021000, valid_loss: 0.017542
SEED: 5, FOLD: 5, SCORED_ONLY, EPOCH: 3, train_loss: 0.020003, valid_loss: 0.017224
SEED: 5, FOLD: 5, SCORED_ONLY, EPOCH: 4, train_loss: 0.021623, valid_loss: 0.016844
SEED: 5, FOLD: 5, SCORED_ONLY, EPOCH: 5, train_loss: 0.019764, valid_loss: 0.016749
SEED: 5, FOLD: 5, SCORED_ONLY, EPOCH: 6, train_loss: 0.019617, valid_loss: 0.016862
SEED: 5, FOLD: 5, SCORED_ONLY, EPOCH: 7, train_loss: 0.019595, valid_los

SEED: 6, FOLD: 0, ALL_TARGETS, EPOCH: 21, train_loss: 0.012142, valid_loss: 0.008914
SEED: 6, FOLD: 0, ALL_TARGETS, EPOCH: 22, train_loss: 0.012049, valid_loss: 0.008889
SEED: 6, FOLD: 0, ALL_TARGETS, EPOCH: 23, train_loss: 0.011990, valid_loss: 0.008899
SEED: 6, FOLD: 0, SCORED_ONLY, EPOCH: 0, train_loss: 0.641852, valid_loss: 0.265799
SEED: 6, FOLD: 0, SCORED_ONLY, EPOCH: 1, train_loss: 0.058865, valid_loss: 0.019819
SEED: 6, FOLD: 0, SCORED_ONLY, EPOCH: 2, train_loss: 0.020959, valid_loss: 0.017381
SEED: 6, FOLD: 0, SCORED_ONLY, EPOCH: 3, train_loss: 0.019960, valid_loss: 0.017015
SEED: 6, FOLD: 0, SCORED_ONLY, EPOCH: 4, train_loss: 0.022041, valid_loss: 0.016842
SEED: 6, FOLD: 0, SCORED_ONLY, EPOCH: 5, train_loss: 0.019783, valid_loss: 0.016672
SEED: 6, FOLD: 0, SCORED_ONLY, EPOCH: 6, train_loss: 0.019617, valid_loss: 0.016737
SEED: 6, FOLD: 0, SCORED_ONLY, EPOCH: 7, train_loss: 0.019560, valid_loss: 0.016641
SEED: 6, FOLD: 0, SCORED_ONLY, EPOCH: 8, train_loss: 0.019744, valid_loss

SEED: 6, FOLD: 2, ALL_TARGETS, EPOCH: 22, train_loss: 0.012080, valid_loss: 0.008819
SEED: 6, FOLD: 2, ALL_TARGETS, EPOCH: 23, train_loss: 0.012025, valid_loss: 0.008808
SEED: 6, FOLD: 2, SCORED_ONLY, EPOCH: 0, train_loss: 0.643925, valid_loss: 0.267102
SEED: 6, FOLD: 2, SCORED_ONLY, EPOCH: 1, train_loss: 0.059320, valid_loss: 0.019645
SEED: 6, FOLD: 2, SCORED_ONLY, EPOCH: 2, train_loss: 0.020992, valid_loss: 0.017374
SEED: 6, FOLD: 2, SCORED_ONLY, EPOCH: 3, train_loss: 0.020060, valid_loss: 0.016854
SEED: 6, FOLD: 2, SCORED_ONLY, EPOCH: 4, train_loss: 0.021669, valid_loss: 0.016689
SEED: 6, FOLD: 2, SCORED_ONLY, EPOCH: 5, train_loss: 0.019818, valid_loss: 0.016663
SEED: 6, FOLD: 2, SCORED_ONLY, EPOCH: 6, train_loss: 0.019721, valid_loss: 0.016513
SEED: 6, FOLD: 2, SCORED_ONLY, EPOCH: 7, train_loss: 0.019604, valid_loss: 0.016417
SEED: 6, FOLD: 2, SCORED_ONLY, EPOCH: 8, train_loss: 0.019823, valid_loss: 0.016562
SEED: 6, FOLD: 2, SCORED_ONLY, EPOCH: 9, train_loss: 0.019587, valid_loss:

SEED: 6, FOLD: 4, ALL_TARGETS, EPOCH: 23, train_loss: 0.011992, valid_loss: 0.008942
SEED: 6, FOLD: 4, SCORED_ONLY, EPOCH: 0, train_loss: 0.644492, valid_loss: 0.262209
SEED: 6, FOLD: 4, SCORED_ONLY, EPOCH: 1, train_loss: 0.058438, valid_loss: 0.019881
SEED: 6, FOLD: 4, SCORED_ONLY, EPOCH: 2, train_loss: 0.021025, valid_loss: 0.017470
SEED: 6, FOLD: 4, SCORED_ONLY, EPOCH: 3, train_loss: 0.020050, valid_loss: 0.016952
SEED: 6, FOLD: 4, SCORED_ONLY, EPOCH: 4, train_loss: 0.023344, valid_loss: 0.016814
SEED: 6, FOLD: 4, SCORED_ONLY, EPOCH: 5, train_loss: 0.019785, valid_loss: 0.016803
SEED: 6, FOLD: 4, SCORED_ONLY, EPOCH: 6, train_loss: 0.019688, valid_loss: 0.016649
SEED: 6, FOLD: 4, SCORED_ONLY, EPOCH: 7, train_loss: 0.019543, valid_loss: 0.016608
SEED: 6, FOLD: 4, SCORED_ONLY, EPOCH: 8, train_loss: 0.019851, valid_loss: 0.016741
SEED: 6, FOLD: 4, SCORED_ONLY, EPOCH: 9, train_loss: 0.019616, valid_loss: 0.016578
SEED: 6, FOLD: 4, SCORED_ONLY, EPOCH: 10, train_loss: 0.019428, valid_loss:

SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 0, train_loss: 0.642887, valid_loss: 0.265419
SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 1, train_loss: 0.058948, valid_loss: 0.019567
SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 2, train_loss: 0.020937, valid_loss: 0.017161
SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 3, train_loss: 0.019939, valid_loss: 0.016775
SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 4, train_loss: 0.023191, valid_loss: 0.016699
SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 5, train_loss: 0.019728, valid_loss: 0.016635
SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 6, train_loss: 0.019618, valid_loss: 0.016564
SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 7, train_loss: 0.019504, valid_loss: 0.016487
SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 8, train_loss: 0.019770, valid_loss: 0.016540
SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 9, train_loss: 0.019514, valid_loss: 0.016468
SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 10, train_loss: 0.019400, valid_loss: 0.016387
SEED: 6, FOLD: 6, SCORED_ONLY, EPOCH: 11, train_loss: 0.019337, valid_loss:

In [32]:
for col in target_cols:
    test[col] = 0
test[target_cols] = predictions

In [33]:
from datetime import timedelta
str(timedelta(seconds=time_diff))

'1:24:01.741661'

In [34]:
train_targets_scored.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
len(target_cols)

206

In [36]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
print('oof shape:',valid_results.shape)
y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0

for i in range(len(target_cols)):
    score += log_loss(y_true[:, i], y_pred[:, i])

print("CV log_loss: ", score / y_pred.shape[1])

#CV log_loss:  0.015612189361251968  retrain

oof shape: (23814, 207)
CV log_loss:  0.014492626992474233


In [37]:
valid_results.shape

(23814, 207)

In [38]:
valid_results.to_csv('oof_model11.csv', index=False)

In [39]:

sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission_model11.csv', index=False)

In [40]:
# sub.shape